# Improving RAG Retrieval (Feature Track 4)

Coded by Adelene Lai (Econetta AG)

26.02.2026

There are several ways to retrieve chunks:
* Dense - distance based on embedding of query
* Sparse - distance based on (key)words
* Hybrid - weighted sum of sparse + dense

Top-k chunks are those closest to the query using ANN.

Let's compare the retrievers for different questions.

In [1]:
from pathlib import Path

from conversational_toolkit.agents.base import QueryWithContext
from conversational_toolkit.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from conversational_toolkit.retriever.vectorstore_retriever import VectorStoreRetriever
from conversational_toolkit.retriever.bm25_retriever import BM25Retriever
from conversational_toolkit.retriever.hybrid_retriever import HybridRetriever
from conversational_toolkit.retriever.reranking_retriever import RerankingRetriever

# from conversational_toolkit.retriever.XYZ import XYZRetriever

from sme_kt_zh_collaboration_rag.feature0_baseline_rag import (
    load_chunks,
    inspect_chunks,
    build_vector_store,
    inspect_retrieval,
    build_agent,
    build_llm,
    ask,
    DATA_DIR,
    VS_PATH,
    EMBEDDING_MODEL,
    RETRIEVER_TOP_K,
)

# Choose your LLM backend: "ollama" (local, requires `ollama serve`) or "openai" (requires OPENAI_API_KEY)
BACKEND = "openai"  # set this before running

if not BACKEND:
    raise ValueError(
        'BACKEND is not set. Edit the line above and set it to "ollama", or "openai".\n'
        "See Renku_README.md for setup instructions."
    )

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


2026-02-26 15:38:05.687125535 [W:onnxruntime:Default, device_discovery.cc:211 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:91 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"
/home/alai/projects/myfork/sme-kt-zh-collaboration-rag/rag_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chunks = load_chunks(5)

embedding_model = SentenceTransformerEmbeddings(model_name=EMBEDDING_MODEL)
vector_store = await build_vector_store(chunks, embedding_model, reset=False)
llm = build_llm("openai", model_name="gpt-4o-mini")

2026-02-26 15:38:13.411 | INFO     | sme_kt_zh_collaboration_rag.feature0_baseline_rag:load_chunks:205 - Chunking 5 files from /home/alai/projects/myfork/sme-kt-zh-collaboration-rag/data


5


2026-02-26 15:38:13.696 | DEBUG    | sme_kt_zh_collaboration_rag.feature0_baseline_rag:load_chunks:217 -   ART_internal_procurement_policy.pdf: 12 chunks
2026-02-26 15:38:13.893 | DEBUG    | sme_kt_zh_collaboration_rag.feature0_baseline_rag:load_chunks:217 -   ART_logylight_incomplete_datasheet.pdf: 6 chunks
2026-02-26 15:38:14.004 | DEBUG    | sme_kt_zh_collaboration_rag.feature0_baseline_rag:load_chunks:217 -   ART_product_catalog.pdf: 7 chunks
2026-02-26 15:38:14.010 | DEBUG    | sme_kt_zh_collaboration_rag.feature0_baseline_rag:load_chunks:217 -   ART_product_overview.xlsx: 1 chunks
2026-02-26 15:38:14.121 | DEBUG    | sme_kt_zh_collaboration_rag.feature0_baseline_rag:load_chunks:217 -   ART_relicyc_logypal1_datasheet_2021.pdf: 5 chunks
2026-02-26 15:38:14.121 | INFO     | sme_kt_zh_collaboration_rag.feature0_baseline_rag:load_chunks:221 - Done, 31 chunks total
2026-02-26 15:38:16.578 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:__init__:57 - Sentence Transfo

## Try with normal VectorStoreRetriever that does dense retrieval

In [3]:
QUERY = "What materials is the Logypal 1 pallet made from?"

results = await inspect_retrieval(
    QUERY, vector_store, embedding_model, top_k=RETRIEVER_TOP_K
)

TypeError: inspect_retrieval() got multiple values for argument 'top_k'

In [4]:
llm = build_llm(backend=BACKEND)

SYSTEM_PROMPT = (
    "You are a helpful AI assistant specialised in sustainability and product compliance for PrimePack AG.\n\n"
    "You will receive document excerpts relevant to the user's question. Produce the best possible answer using only the information in those excerpts."
)
agent = build_agent(
    llm=llm,
    top_k=RETRIEVER_TOP_K,
    system_prompt=SYSTEM_PROMPT,
    number_query_expansion=0,
    retriever=VectorStoreRetriever(
        embedding_model=embedding_model,
        vector_store=vector_store,
        top_k=RETRIEVER_TOP_K,
    ),  # 0 = no expansion; see Feature Track 3 for more
)
print("RAG agent assembled.")

2026-02-26 15:38:33.878 | INFO     | sme_kt_zh_collaboration_rag.feature0_baseline_rag:build_llm:140 - LLM backend: OpenAI (gpt-4o-mini)
2026-02-26 15:38:33.898 | DEBUG    | conversational_toolkit.llms.openai:__init__:63 - OpenAI LLM loaded: gpt-4o-mini; temperature: 0.3; seed: 42; tools: None; tool_choice: None; response_format: {'type': 'text'}
2026-02-26 15:38:33.899 | INFO     | sme_kt_zh_collaboration_rag.feature0_baseline_rag:build_agent:349 - RAG agent ready (top_k=5  query_expansion=0)


RAG agent assembled.


In [5]:
QUERY = "What materials is the Logypal 1 pallet made from?"

print("---------------------------")
print(f"Query: {QUERY!r}")
print("---------------------------")
answer = await ask(agent, QUERY)

2026-02-26 15:38:36.913 | INFO     | sme_kt_zh_collaboration_rag.feature0_baseline_rag:ask:366 - Query: 'What materials is the Logypal 1 pallet made from?'


---------------------------
Query: 'What materials is the Logypal 1 pallet made from?'
---------------------------


2026-02-26 15:38:37.191 | DEBUG    | conversational_toolkit.embeddings.sentence_transformer:get_embeddings:76 - sentence-transformers/all-MiniLM-L6-v2 embeddings size: (1, 384)
2026-02-26 15:38:38.638 | INFO     | sme_kt_zh_collaboration_rag.feature0_baseline_rag:ask:369 - Answer:


The Logypal 1 pallet is made from 100% post-consumer recycled plastic, primarily sourced from end-of-life agricultural packaging, such as silage film, and industrial packaging waste.
Sources (5):
  'ART_relicyc_logypal1_datasheet_2021.pdf'  |  '## Overview'
  'ART_logylight_incomplete_datasheet.pdf'  |  '## Product Overview'
  'ART_logylight_incomplete_datasheet.pdf'  |  '## Material Composition'
  'ART_relicyc_logypal1_datasheet_2021.pdf'  |  '## Material Composition (2021)'
  'ART_logylight_incomplete_datasheet.pdf'  |  '# Relicyc LogyLight — Sustainability Information Sheet'


In [6]:
agent = build_agent(
    llm=llm,
    top_k=RETRIEVER_TOP_K,
    system_prompt=SYSTEM_PROMPT,
    number_query_expansion=0,
    retriever=BM25Retriever(vector_store=vector_store, top_k=RETRIEVER_TOP_K),
)
print("RAG agent assembled.")

AttributeError: 'str' object has no attribute 'content'